In [1]:
import rdflib

In [2]:
from rdflib import Namespace

In [3]:
owlNS = Namespace("http://www.w3.org/2002/07/owl#")
owlClass = owlNS["Class"]
owlObjectProperty = owlNS["ObjectProperty"]
owlDatatypeProperty = owlNS["DatatypeProperty"]
rdfNS = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfProperty = rdfNS["Property"]
rdfType = rdfNS["type"]
rdfsNS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
rdfsSubClassOf = rdfsNS["subClassOf"]
rdfsDomain = rdfsNS["domain"]
rdfsRange = rdfsNS["range"]
xsdNS = Namespace("http://www.w3.org/2001/XMLSchema#")
xsdString = xsdNS["string"]

In [7]:
filmNS = Namespace("http://www.semprog.com/film#")

In [8]:
objectClass = filmNS['Object']
personClass = filmNS['Person']
filmClass = filmNS['Film']
performanceClass = filmNS['Performance']
actorClass = filmNS['Actor']
roleClass = filmNS['Role']
directorClass = filmNS['Director']

In [9]:
name = filmNS['name']
starring = filmNS['starring']
hasActor = filmNS['has_actor']
hasRole = filmNS['has_role']
directedBy = filmNS['directed_by']

In [10]:
classTriples = [
# class declarations:
(filmNS['Object'], rdfType, owlClass),
(filmNS['Person'], rdfType, owlClass),
(filmNS['Film'], rdfType, owlClass),
(filmNS['Actor'], rdfType, owlClass),
(filmNS['Role'], rdfType, owlClass),
(filmNS['Director'], rdfType, owlClass),
(filmNS['Performance'], rdfType, owlClass)
]

In [11]:
classHeirarchyTriples = [
# class heirarchy:
(filmNS['Film'], rdfsSubClassOf, filmNS['Object']),
(filmNS['Person'], rdfsSubClassOf, filmNS['Object']),
(filmNS['Actor'], rdfsSubClassOf, filmNS['Person']),
(filmNS['Role'], rdfsSubClassOf, filmNS['Object']),
(filmNS['Director'], rdfsSubClassOf, filmNS['Person'])
]

In [12]:
propertyTriples = [
# name property:
(filmNS['name'], rdfType, owlDatatypeProperty),
(filmNS['name'], rdfsDomain, filmNS['Object']),
(filmNS['name'], rdfsRange, xsdString),
# starring property:
(filmNS['starring'], rdfType, owlObjectProperty),
(filmNS['starring'], rdfsDomain, filmNS['Film']),
(filmNS['starring'], rdfsRange, filmNS['Performance']),
# hasActor property:
(filmNS['hasActor'], rdfType, owlObjectProperty),
(filmNS['hasActor'], rdfsDomain, filmNS['Performance']),
(filmNS['hasActor'], rdfsRange, filmNS['Actor']),
# hasRole property:
(filmNS['hasRole'], rdfType, owlObjectProperty),
(filmNS['hasRole'], rdfsDomain, filmNS['Performance']),
(filmNS['hasRole'], rdfsRange, filmNS['Role']),
# directedBy property:
(filmNS['directedBy'], rdfType, owlObjectProperty),
(filmNS['directedBy'], rdfsDomain, filmNS['Film']),
(filmNS['directedBy'], rdfsRange, filmNS['Director'])
]

In [13]:
from rdflib import ConjunctiveGraph
graph = ConjunctiveGraph()
for triple in classTriples: graph.add(triple)
for triple in classHeirarchyTriples: graph.add(triple)
for triple in propertyTriples: graph.add(triple)

In [14]:
list(graph.subjects(rdfType, owlClass))

[rdflib.term.URIRef('http://www.semprog.com/film#Person'),
 rdflib.term.URIRef('http://www.semprog.com/film#Object'),
 rdflib.term.URIRef('http://www.semprog.com/film#Actor'),
 rdflib.term.URIRef('http://www.semprog.com/film#Role'),
 rdflib.term.URIRef('http://www.semprog.com/film#Film'),
 rdflib.term.URIRef('http://www.semprog.com/film#Performance'),
 rdflib.term.URIRef('http://www.semprog.com/film#Director')]

In [15]:
list(graph.subjects(rdfType, owlObjectProperty))

[rdflib.term.URIRef('http://www.semprog.com/film#hasRole'),
 rdflib.term.URIRef('http://www.semprog.com/film#starring'),
 rdflib.term.URIRef('http://www.semprog.com/film#hasActor'),
 rdflib.term.URIRef('http://www.semprog.com/film#directedBy')]

In [19]:
def isSubClassOf(subClass, superClass, graph):
    if subClass == superClass: return True
    for parentClass in graph.objects(subClass, rdfsSubClassOf):
        if isSubClassOf(parentClass, superClass, graph):
            return True
        else: 
            return False

In [20]:
isSubClassOf(filmNS['Actor'], filmNS['Person'], graph)

True

In [22]:
isSubClassOf(filmNS['Film'], filmNS['Person'], graph)

False

In [23]:
from rdflib import Literal
filmTriples = [
# movie:
(filmNS['blade_runner'], rdfType, filmNS['Film']),
(filmNS['blade_runner'], filmNS['name'],Literal("Blade Runner", datatype=xsdString)),
(filmNS['blade_runner'], filmNS['starring'], filmNS['bl_performance']),
(filmNS['blade_runner'], filmNS['directedBy'], filmNS['ridley_scott']),
# performance:
(filmNS['bl_performance'], rdfType, filmNS['Performance']),
(filmNS['bl_performance'], filmNS['hasActor'], filmNS['harrison_ford']),
(filmNS['bl_performance'], filmNS['hasRole'], filmNS['rick_deckard']),
# actor:
(filmNS['harrison_ford'], rdfType, filmNS['Actor']),
(filmNS['harrison_ford'], filmNS['name'],Literal("Harrison Ford", datatype=xsdString)),
# role:
(filmNS['rick_deckard'], rdfType, filmNS['Role']),
(filmNS['rick_deckard'], filmNS['name'],Literal("Rick Deckard", datatype=xsdString)),
# director:
(filmNS['ridley_scott'], rdfType, filmNS['Director']),
(filmNS['ridley_scott'], filmNS['name'],Literal("Ridley Scott", datatype=xsdString))
]
for triple in filmTriples: graph.add(triple)

In [25]:
def findInstances(queryClass, graph, instances=None):
    if instances is None: instances = set()
    for instance in graph.subjects(rdfType, queryClass): instances.add(instance)
    for subClass in graph.subjects(rdfsSubClassOf, queryClass):
        findInstances(subClass, graph, instances)
    return instances

In [26]:
findInstances(personClass, graph)

{rdflib.term.URIRef('http://www.semprog.com/film#harrison_ford'),
 rdflib.term.URIRef('http://www.semprog.com/film#ridley_scott')}

In [28]:
graph.serialize(destination='output.ttl', format='turtle')